In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip


In [21]:
import tensorflow
import gensim

In [22]:
import pandas as pd

train_df = pd.read_csv( '/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test_df = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')

train_df.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [23]:
from nltk.tokenize import WordPunctTokenizer
from collections import Counter
from string import punctuation, ascii_lowercase
import regex as re
from tqdm import tqdm

# replace urls
re_url = re.compile(r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\
                    .([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*",
                    re.MULTILINE|re.UNICODE)
# replace ips
re_ip = re.compile("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}")

# setup tokenizer
tokenizer = WordPunctTokenizer()

vocab = Counter()

def text_to_wordlist(text, lower=False):
    # replace URLs
    text = re_url.sub("URL", text)
    
    # replace IPs
    text = re_ip.sub("IPADDRESS", text)
    
    # Tokenize
    text = tokenizer.tokenize(text)
    
    # optional: lower case
    if lower:
        text = [t.lower() for t in text]
    
    # Return a list of words
    vocab.update(text)
    return text

def process_comments(list_sentences, lower=False):
    comments = []
    for text in tqdm(list_sentences):
        txt = text_to_wordlist(text, lower=lower)
        comments.append(txt)
    return comments


list_sentences_train = list(train_df["comment_text"].fillna("NAN_WORD").values)
list_sentences_test = list(test_df["comment_text"].fillna("NAN_WORD").values)

comments = process_comments(list_sentences_train + list_sentences_test, lower=True)

100%|██████████| 312735/312735 [00:19<00:00, 15660.72it/s]


In [ ]:
from gensim.models import Word2Vec
model_cbow = Word2Vec(comments, vector_size =100, window=5, min_count=5, workers=16, sg=0, negative=5) # if sg=1, we use Skip-gram

In [25]:
model_cbow.wv.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])

[('prince', 0.9814999103546143),
 ('princess', 0.9668576717376709),
 ('queen', 0.951403021812439),
 ('mary', 0.9439194798469543),
 ('bishop', 0.9320930242538452),
 ('mayor', 0.9300081133842468),
 ('abraham', 0.9268338680267334),
 ('duchess', 0.9258257746696472),
 ('hrh', 0.9180528521537781),
 ('victoria', 0.9164304137229919)]

In [26]:
MAX_NB_WORDS = len(model_cbow.wv.key_to_index)
MAX_SEQUENCE_LENGTH = 200

from tensorflow.keras.preprocessing.sequence import pad_sequences

word_index = {t[0]: i+1 for i,t in enumerate(vocab.most_common(MAX_NB_WORDS))}
sequences = [[word_index.get(t, 0) for t in comment]
             for comment in comments[:len(list_sentences_train)]]
test_sequences = [[word_index.get(t, 0)  for t in comment] 
                  for comment in comments[len(list_sentences_train):]]

# pad
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, 
                     padding="pre", truncating="post")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train_df[list_classes].values
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y.shape)

test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="pre",
                          truncating="post")
print('Shape of test_data tensor:', test_data.shape)

Shape of data tensor: (159571, 200)
Shape of label tensor: (159571, 6)
Shape of test_data tensor: (153164, 200)


In [27]:
import numpy as np
WV_DIM = 100
nb_words = min(MAX_NB_WORDS, len(model_cbow.wv.key_to_index))
# 我们首先先随机初始化嵌入矩阵
wv_matrix = (np.random.rand(nb_words, WV_DIM) - 0.5) / 5.0
# 然后尝试地将已经学习好的词向量覆盖掉矩阵的对应元素
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    try :
        embedding_vector = model_cbow.wv.get_vector(word)
        wv_matrix[i] = embedding_vector
    except:
        print('can not find word', word)
        pass     

In [28]:
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout,SpatialDropout1D, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization

wv_layer = Embedding(nb_words,
                     WV_DIM,
                     mask_zero=False,
                     weights=[wv_matrix], # 在这里，我们用之前构建的wv_matrix作为初始参数的设定
                     input_length=MAX_SEQUENCE_LENGTH,
                     trainable=False)

# Inputs
comment_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = wv_layer(comment_input)

# biLSTM
embedded_sequences = SpatialDropout1D(0.2)(embedded_sequences)
x = Bidirectional(LSTM(64, return_sequences=False))(embedded_sequences)

# Output
x = Dropout(0.2)(x)
x = BatchNormalization()(x)
preds = Dense(6, activation='sigmoid')(x)

# build the model
model = Model(inputs=[comment_input], outputs=preds)
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.001, clipnorm=.25, beta_1=0.7, beta_2=0.99),
              metrics=[])
              
hist = model.fit([data], y, validation_split=0.1, epochs=10, batch_size=256, shuffle=True)

In [29]:
y_test = model.predict(test_data)
y_test

In [30]:
y_test = pd.DataFrame(round(y_test))

In [31]:
submission = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')
submission.iloc[:,1:] = y_test
submission.to_csv('/kaggle/working/submission.csv', index=False)

In [32]:
# wv_layer = Embedding(nb_words,
#                      WV_DIM,
#                      mask_zero=False,
#                      # weights=[wv_matrix],
#                      input_length=MAX_SEQUENCE_LENGTH,
#                      trainable=False)

# # Inputs
# comment_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
# embedded_sequences = wv_layer(comment_input)

# # biLSTM
# embedded_sequences = SpatialDropout1D(0.2)(embedded_sequences)
# x = Bidirectional(LSTM(64, return_sequences=False))(embedded_sequences)

# # Output
# x = Dropout(0.2)(x)
# x = BatchNormalization()(x)
# preds = Dense(6, activation='sigmoid')(x)

# # build the model
# model = Model(inputs=[comment_input], outputs=preds)
# model.compile(loss='binary_crossentropy',
#               optimizer=Adam(learning_rate=0.001, clipnorm=.25, beta_1=0.7, beta_2=0.99),
#               metrics=[])

# hist_ = model.fit([data], y, validation_split=0.1,
#                  epochs=10, batch_size=256, shuffle=True)

Epoch 1/10
561/561 [==============================] - 22s 33ms/step - loss: 0.1792 - val_loss: 0.1244
Epoch 2/10
561/561 [==============================] - 18s 32ms/step - loss: 0.1073 - val_loss: 0.1045
Epoch 3/10
561/561 [==============================] - 18s 32ms/step - loss: 0.1043 - val_loss: 0.0996
Epoch 4/10
561/561 [==============================] - 18s 32ms/step - loss: 0.0988 - val_loss: 0.0972
Epoch 5/10
561/561 [==============================] - 18s 32ms/step - loss: 0.0934 - val_loss: 0.0915
Epoch 6/10
561/561 [==============================] - 18s 32ms/step - loss: 0.0885 - val_loss: 0.0857
Epoch 7/10
561/561 [==============================] - 18s 32ms/step - loss: 0.0857 - val_loss: 0.0826
Epoch 8/10
561/561 [==============================] - 18s 32ms/step - loss: 0.0829 - val_loss: 0.0834
Epoch 9/10
561/561 [==============================] - 18s 32ms/step - loss: 0.0806 - val_loss: 0.0876
Epoch 10/10
561/561 [==============================] - 18s 32ms/step - loss: 0.078

In [33]:
# y_test = model.predict(test_data)

4787/4787 [==============================] - 34s 7ms/step


In [37]:
# y_test = pd.DataFrame(np.round(y_test))

In [39]:
# submission = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')
# submission.iloc[:,1:] = y_test
# submission.to_csv('/kaggle/working/submission2.csv', index=False)